**About** : This notebook is used to validate models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import timm
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *
from numerize.numerize import numerize

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    init_neptune
)

from params import *
from util.plots import *
from data.dataset import *
from data.transforms import get_transfos
from data.preparation import *
from util.torch import init_distributed, count_parameters, load_model_weights, count_parameters
from util.plots import plot_sample
from model_zoo.models import define_model


from training.main import k_fold

## Data

In [ ]:
LOG_FOLDER = "../logs/2023-07-03/30/"
dices = json.load(open(LOG_FOLDER + "dices.json", "r"))

In [ ]:
th, dice = max(dices.items(), key=operator.itemgetter(1))

th, dice
th = float(th)

In [ ]:
plt.plot(np.array(list(dices.keys())).astype(float), dices.values())
plt.axvline(th, c="salmon")
plt.xlim(th - 0.1, th + 0.1)
plt.ylim(dice - 0.01, dice + 0.002)
plt.title(f'dice={dice:.3f}, th={th:.2f}')
plt.show()

In [ ]:
# df_split = prepare_folds()

In [ ]:
df = prepare_data(DATA_PATH, processed_folder="false_color/")

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(DATA_PATH + "folds_4.csv")
    df = df.merge(folds)

In [ ]:
df['fold'].value_counts()

### Dataset

In [ ]:
df = df[df['has_contrail']].reset_index(drop=True)

In [ ]:
transfos =  get_transfos(strength=0)
dataset = ContrailDataset(df, transfos, use_soft_mask=True)

transfos

In [ ]:
for idx in np.random.choice(len(dataset), 3):
    idx = 4
    image, mask, y = dataset[idx]
    plot_sample(image.numpy().transpose(1, 2, 0), mask.numpy().transpose(1, 2, 0))

    break

Done ! 